In [52]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
sp = stopwords.words('english')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
df = pd.read_csv('/content/drive/MyDrive/Spam Message Detection/Data/spam.csv', encoding='ISO-8859-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


Drop The unusual Columns

In [14]:
df = df.drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'])

In [15]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

There is a imbalanced in dataframe

#Start the cleaning process

In [17]:
df.shape

(5572, 2)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


There is no NULL values

In [24]:
df.duplicated().sum()

403

In [22]:
df[df['v1'] == 'spam'].duplicated().sum()

94

Remove Duplicate Data

In [27]:
df = df.drop_duplicates(keep = 'first')

In [30]:
df.duplicated().sum()

0

In [33]:
from sklearn.preprocessing import LabelEncoder

In [35]:
lb = LabelEncoder()
df['v1'] = lb.fit_transform(df['v1'])

In [36]:
df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [37]:
df['num_char'] = df['v2'].apply(len)

In [42]:
df['num_words'] = df['v2'].apply(lambda x: len(nltk.word_tokenize(x)))

In [43]:
df.head()

,v1,v2,num_char,num_words
0,0,"Go until jurong point, crazy.. Available only ...",111,24
1,0,Ok lar... Joking wif u oni...,29,8
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,37
3,0,U dun say so early hor... U c already then say...,49,13
4,0,"Nah I don't think he goes to usf, he lives aro...",61,15


In [46]:
df[df['v1'] == 0][['num_char', 'num_words']].describe()

,num_char,num_words
count,4516.000000,4516.000000
mean,70.459256,17.123782
std,56.358207,13.493970
min,2.000000,1.000000
25%,34.000000,8.000000
50%,52.000000,13.000000
75%,90.000000,22.000000
max,910.000000,220.000000


In [47]:
df[df['v1'] == 1][['num_char', 'num_words']].describe()

,num_char,num_words
count,653.000000,653.000000
mean,137.891271,27.667688
std,30.137753,7.008418
min,13.000000,2.000000
25%,132.000000,25.000000
50%,149.000000,29.000000
75%,157.000000,32.000000
max,224.000000,46.000000


In [48]:
df.corr()

<ipython-input-48-2f6f6606aa2c>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr()


,v1,num_char,num_words
v1,1.000000,0.384717,0.262912
num_char,0.384717,1.000000,0.965760
num_words,0.262912,0.965760,1.000000


In [51]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [53]:
def Cleaning_NLP(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  y = []
  for i in text:
    if i.isalnum():
      y.append(i)

  text = y[:]
  y.clear()
  for i in text:
    if i not in sp and i not in string.punctuation:
      y.append(i)

  text = y[:]
  y.clear()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

In [55]:
df['v2'] = df['v2'].apply(Cleaning_NLP)

In [57]:
df.head()

,v1,v2,num_char,num_words
0,0,go jurong point crazi avail bugi n great world...,111,24
1,0,ok lar joke wif u oni,29,8
2,1,free entri 2 wkli comp win fa cup final tkt 21...,155,37
3,0,u dun say earli hor u c alreadi say,49,13
4,0,nah think goe usf live around though,61,15


#Data Modeling Starts

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x = cv.fit_transform(df['v2']).toarray()

In [62]:
y = df['v1'].values

In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 50)

In [64]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [65]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [66]:
gnb.fit(x_train, y_train)
y_pred1 = gnb.predict(x_test)
print(accuracy_score(y_test, y_pred1))
print(precision_score(y_test, y_pred1))

0.874274661508704
0.5084033613445378


In [68]:
mnb.fit(x_train, y_train)
y_pred2 = mnb.predict(x_test)
print(accuracy_score(y_test, y_pred2))
print(precision_score(y_test, y_pred2))

0.9729206963249516
0.8680555555555556


In [69]:
bnb.fit(x_train, y_train)
y_pred3 = bnb.predict(x_test)
print(accuracy_score(y_test, y_pred3))
print(precision_score(y_test, y_pred3))

0.971953578336557
0.981651376146789


The accuracy and precision score is good but we can try for better with TFIDF Vectorizer

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features = 3000)
x1 = tfidf.fit_transform(df['v2']).toarray()

In [72]:
y1 = df['v1'].values

In [79]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size = 0.2, random_state = 2)

In [80]:
gnb.fit(x_train1, y_train1)
y_pred11 = gnb.predict(x_test1)
print(accuracy_score(y_test1, y_pred11))
print(precision_score(y_test1, y_pred11))

0.8694390715667312
0.5068493150684932


In [81]:
mnb.fit(x_train1, y_train1)
y_pred12 = mnb.predict(x_test1)
print(accuracy_score(y_test1, y_pred12))
print(precision_score(y_test1, y_pred12))

0.9709864603481625
1.0


In [82]:
bnb.fit(x_train1, y_train1)
y_pred13 = bnb.predict(x_test1)
print(accuracy_score(y_test1, y_pred13))
print(precision_score(y_test1, y_pred13))

0.9835589941972921
0.991869918699187


In [85]:
import pickle
pickle.dump(tfidf,open('/content/drive/MyDrive/Spam Message Detection/vectorizer (1).pkl','wb'))
pickle.dump(mnb,open('/content/drive/MyDrive/Spam Message Detection/model (1).pkl','wb'))

#Deploy on Streamlit

In [83]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.6 MB/s eta 0:00:00


In [86]:
%%writefile app.py

import streamlit as st
import pickle
import string
from nltk.corpus import stopwords
import nltk
from nltk.stem.porter import PorterStemmer


cv = pickle.load(open('/content/drive/MyDrive/Spam Message Detection/vectorizer (1).pkl','rb'))
model = pickle.load(open('/content/drive/MyDrive/Spam Message Detection/model (1).pkl','rb'))

ps = PorterStemmer()

def transform_text(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  y = []
  for i in text:
    if i.isalnum():
      y.append(i)

  text = y[:]
  y.clear()
  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)

  text = y[:]
  y.clear()
  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

st.title("Detect The Spam Messages")

input_sms = st.text_area("Enter the message")

if st.button('Answer'):

    transformed_sms = transform_text(input_sms)

    vector_input = cv.transform([transformed_sms])

    result = model.predict(vector_input)[0]

    if result == 1:
        st.header("Spam")
    else:
        st.header("Not Spam")

Writing app.py


In [87]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 1.521s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [88]:
!streamlit run app.py &>/content/logs.txt & curl ipv4.icanhazip.com

35.245.120.228


In [90]:
#!npx localtunnel --port 8501